## Cleaning the Data Set

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [2]:
df = pd.read_csv("players_data.csv")
df

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0
0,SR Tendulkar (IND),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20
1,KC Sangakkara (Asia/ICC/SL),2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,15
2,V Kohli (IND),2008-2025,302,290,45,14181,183,57.88,15192,93.34,51,74,16
3,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20
4,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,DS Airee (NEP),2018-2025,64,60,7,1066,105,20.11,1472,72.41,1,4,8
496,DS Smith (WI),2003-2013,47,45,2,1059,107,24.62,1506,70.31,1,5,4
497,JK Kamande (KENYA),1999-2011,86,72,11,1055,74,17.29,1685,62.61,0,3,8
498,IDS Smith (NZ),1980-1992,98,77,16,1055,62*,17.29,1061,99.43,0,3,7


In [3]:
total_cells = df.size
total_missing = df.isnull().sum().sum()

overall_missing_percent = (total_missing / total_cells) * 100
print(f"Overall Missing Values: {overall_missing_percent:.2f}%")


Overall Missing Values: 0.00%


In [4]:
df.shape

(500, 13)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  500 non-null    object 
 1   Span    500 non-null    object 
 2   Mat     500 non-null    int64  
 3   Inns    500 non-null    int64  
 4   NO      500 non-null    int64  
 5   Runs    500 non-null    int64  
 6   HS      500 non-null    object 
 7   Ave     500 non-null    float64
 8   BF      500 non-null    int64  
 9   SR      500 non-null    float64
 10  100     500 non-null    int64  
 11  50      500 non-null    int64  
 12  0       500 non-null    int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 50.9+ KB


In [6]:
df.drop(columns = "0",inplace = True)

In [7]:
df.columns

Index(['Player', 'Span', 'Mat', 'Inns', 'NO', 'Runs', 'HS', 'Ave', 'BF', 'SR',
       '100', '50'],
      dtype='object')

In [8]:
df = df.rename(columns={
    "Player": "Player_Name",
    "Mat": "Matches",
    "Inns": "Innings",
    "Runs": "Total_Runs",
    "Ave": "Batting_Average",
    "SR": "Strike_Rate",
    "NO":"Not_Out",
    "HS":"Highest_Score",
    "BF":"Ball_Faced",
    "100":"100's",
    "50":"50's"
})
df.head()


,Player_Name,Span,Matches,Innings,Not_Out,Total_Runs,Highest_Score,Batting_Average,Ball_Faced,Strike_Rate,100's,50's
0,SR Tendulkar (IND),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96
1,KC Sangakkara (Asia/ICC/SL),2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93
2,V Kohli (IND),2008-2025,302,290,45,14181,183,57.88,15192,93.34,51,74
3,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82
4,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68


In [9]:
df[['Player_Names','Country(Region)']] = df['Player_Name'].str.rsplit('(',n=1,expand = True)

In [10]:
df["Country(Region)"] = df["Country(Region)"].str.replace(")","")

In [11]:
df.drop(columns = "Player_Name",inplace = True)

In [12]:
cols = ['Player_Names', 'Country(Region)'] + [c for c in df.columns if c not in ['Player_Names', 'Country(Region)']]
df = df[cols]

df.head()

,Player_Names,Country(Region),Span,Matches,Innings,Not_Out,Total_Runs,Highest_Score,Batting_Average,Ball_Faced,Strike_Rate,100's,50's
0,SR Tendulkar,IND,1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96
1,KC Sangakkara,Asia/ICC/SL,2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93
2,V Kohli,IND,2008-2025,302,290,45,14181,183,57.88,15192,93.34,51,74
3,RT Ponting,AUS/ICC,1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82
4,ST Jayasuriya,Asia/SL,1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68


In [13]:
df.duplicated().sum()

50

In [15]:
df = df.drop_duplicates()

In [16]:
df.duplicated().sum()

0

In [17]:
df.isnull().sum()

Player_Names       0
Country(Region)    0
Span               0
Matches            0
Innings            0
Not_Out            0
Total_Runs         0
Highest_Score      0
Batting_Average    0
Ball_Faced         0
Strike_Rate        0
100's              0
50's               0
dtype: int64

In [18]:
df.shape

(450, 13)

In [19]:
df.head(10)

,Player_Names,Country(Region),Span,Matches,Innings,Not_Out,Total_Runs,Highest_Score,Batting_Average,Ball_Faced,Strike_Rate,100's,50's
0,SR Tendulkar,IND,1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96
1,KC Sangakkara,Asia/ICC/SL,2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93
2,V Kohli,IND,2008-2025,302,290,45,14181,183,57.88,15192,93.34,51,74
3,RT Ponting,AUS/ICC,1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82
4,ST Jayasuriya,Asia/SL,1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68
5,DPMD Jayawardene,Asia/SL,1998-2015,448,418,39,12650,144,33.37,16020,78.96,19,77
6,Inzamam-ul-Haq,Asia/PAK,1991-2007,378,350,53,11739,137*,39.52,15812,74.24,10,83
7,JH Kallis,Afr/ICC/SA,1996-2014,328,314,53,11579,139,44.36,15885,72.89,17,86
8,SC Ganguly,Asia/IND,1992-2007,311,300,23,11363,183,41.02,15416,73.70,22,72
9,RG Sharma,IND,2007-2025,273,265,36,11168,264,48.76,12034,92.80,32,58


In [20]:
df["Country(Region)"].unique()

array(['IND', 'Asia/ICC/SL', 'AUS/ICC', 'Asia/SL', 'Asia/PAK',
       'Afr/ICC/SA', 'Asia/IND', 'Asia/ICC/IND', 'ICC/WI', 'SL', 'Afr/SA',
       'PAK', 'WI', 'NZ', 'AUS', 'BAN', 'SA', 'Asia/ICC/PAK', 'ICC/NZ',
       'ENG/IRE', 'ENG', 'ZIM', 'IRE', 'ENG/ICC', 'Afr/ZIM', 'AFG',
       'SCOT', 'Asia/BAN', 'Afr/KENYA', 'AUS/SA', 'USA', 'NAM', 'NED',
       'KENYA', 'PNG', 'CAN', 'UAE', 'NEP', 'OMA', 'AUS/NZ'], dtype=object)

In [21]:

df["Country(Region)"] = df["Country(Region)"].str.replace(r"(Asia|Afr|ICC)/", "", regex=True)

replace_dict = {
    "IND": "India",
    "AUS": "Australia",
    "SL": "Sri Lanka",
    "PAK": "Pakistan",
    "SA": "South Africa",
    "WI": "West Indies",
    "NZ": "New Zealand",
    "BAN": "Bangladesh",
    "ENG": "England",
    "IRE": "Ireland",
    "ZIM": "Zimbabwe",
    "AFG": "Afghanistan",
    "SCOT": "Scotland",
    "KENYA": "Kenya",
    "USA": "United States",
    "NAM": "Namibia",
    "NED": "Netherlands",
    "PNG": "Papua New Guinea",
    "CAN": "Canada",
    "UAE": "United Arab Emirates",
    "NEP": "Nepal",
    "OMA": "Oman",
    "AUS/ICC":"Australia",
    "ENG/ICC":"England",
    "AUS/SA":"Australia/South Africa",
    "AUS/NZ":"Australia/New Zealand",
    'ENG/IRE':'England/Ireland'
}

df["Country(Region)"] = df["Country(Region)"].replace(replace_dict)

df["Country(Region)"].unique()


array(['India', 'Sri Lanka', 'Australia', 'Pakistan', 'South Africa',
       'West Indies', 'New Zealand', 'Bangladesh', 'England/Ireland',
       'England', 'Zimbabwe', 'Ireland', 'Afghanistan', 'Scotland',
       'Kenya', 'Australia/South Africa', 'United States', 'Namibia',
       'Netherlands', 'Papua New Guinea', 'Canada',
       'United Arab Emirates', 'Nepal', 'Oman', 'Australia/New Zealand'],
      dtype=object)

In [22]:
df.head(10)

,Player_Names,Country(Region),Span,Matches,Innings,Not_Out,Total_Runs,Highest_Score,Batting_Average,Ball_Faced,Strike_Rate,100's,50's
0,SR Tendulkar,India,1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96
1,KC Sangakkara,Sri Lanka,2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93
2,V Kohli,India,2008-2025,302,290,45,14181,183,57.88,15192,93.34,51,74
3,RT Ponting,Australia,1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82
4,ST Jayasuriya,Sri Lanka,1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68
5,DPMD Jayawardene,Sri Lanka,1998-2015,448,418,39,12650,144,33.37,16020,78.96,19,77
6,Inzamam-ul-Haq,Pakistan,1991-2007,378,350,53,11739,137*,39.52,15812,74.24,10,83
7,JH Kallis,South Africa,1996-2014,328,314,53,11579,139,44.36,15885,72.89,17,86
8,SC Ganguly,India,1992-2007,311,300,23,11363,183,41.02,15416,73.70,22,72
9,RG Sharma,India,2007-2025,273,265,36,11168,264,48.76,12034,92.80,32,58


In [23]:
df = df.rename(columns = {'Country(Region)':'Country'})

In [24]:
df[['Start_Year', 'End_Year']] = df['Span'].str.split('-', expand=True)

In [25]:
df

,Player_Names,Country,Span,Matches,Innings,Not_Out,Total_Runs,Highest_Score,Batting_Average,Ball_Faced,Strike_Rate,100's,50's,Start_Year,End_Year
0,SR Tendulkar,India,1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,1989,2012
1,KC Sangakkara,Sri Lanka,2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,2000,2015
2,V Kohli,India,2008-2025,302,290,45,14181,183,57.88,15192,93.34,51,74,2008,2025
3,RT Ponting,Australia,1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,1995,2012
4,ST Jayasuriya,Sri Lanka,1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,1989,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,DS Airee,Nepal,2018-2025,64,60,7,1066,105,20.11,1472,72.41,1,4,2018,2025
496,DS Smith,West Indies,2003-2013,47,45,2,1059,107,24.62,1506,70.31,1,5,2003,2013
497,JK Kamande,Kenya,1999-2011,86,72,11,1055,74,17.29,1685,62.61,0,3,1999,2011
498,IDS Smith,New Zealand,1980-1992,98,77,16,1055,62*,17.29,1061,99.43,0,3,1980,1992


In [26]:
df.head()

,Player_Names,Country,Span,Matches,Innings,Not_Out,Total_Runs,Highest_Score,Batting_Average,Ball_Faced,Strike_Rate,100's,50's,Start_Year,End_Year
0,SR Tendulkar,India,1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,1989,2012
1,KC Sangakkara,Sri Lanka,2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,2000,2015
2,V Kohli,India,2008-2025,302,290,45,14181,183,57.88,15192,93.34,51,74,2008,2025
3,RT Ponting,Australia,1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,1995,2012
4,ST Jayasuriya,Sri Lanka,1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,1989,2011


In [27]:
df['Highest_Score'] = df['Highest_Score'].astype(str).str.replace('*', '', regex=False)
df['Highest_Score'] = pd.to_numeric(df['Highest_Score'])

In [28]:
df.head()

,Player_Names,Country,Span,Matches,Innings,Not_Out,Total_Runs,Highest_Score,Batting_Average,Ball_Faced,Strike_Rate,100's,50's,Start_Year,End_Year
0,SR Tendulkar,India,1989-2012,463,452,41,18426,200,44.83,21368,86.23,49,96,1989,2012
1,KC Sangakkara,Sri Lanka,2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,2000,2015
2,V Kohli,India,2008-2025,302,290,45,14181,183,57.88,15192,93.34,51,74,2008,2025
3,RT Ponting,Australia,1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,1995,2012
4,ST Jayasuriya,Sri Lanka,1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,1989,2011


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 450 entries, 0 to 499
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Player_Names     450 non-null    object 
 1   Country          450 non-null    object 
 2   Span             450 non-null    object 
 3   Matches          450 non-null    int64  
 4   Innings          450 non-null    int64  
 5   Not_Out          450 non-null    int64  
 6   Total_Runs       450 non-null    int64  
 7   Highest_Score    450 non-null    int64  
 8   Batting_Average  450 non-null    float64
 9   Ball_Faced       450 non-null    int64  
 10  Strike_Rate      450 non-null    float64
 11  100's            450 non-null    int64  
 12  50's             450 non-null    int64  
 13  Start_Year       450 non-null    object 
 14  End_Year         450 non-null    object 
dtypes: float64(2), int64(8), object(5)
memory usage: 56.2+ KB


In [30]:
new_order = [
    'Player_Names', 'Country', 'Start_Year', 'End_Year', 
    'Matches', 'Innings', 'Not_Out', 'Total_Runs', 
    'Highest_Score', 'Batting_Average', 'Ball_Faced', 
    'Strike_Rate', "100's", "50's"
]

df = df[[col for col in new_order if col in df.columns]]

In [31]:
df['Player_Names'].str.upper()

0       SR TENDULKAR 
1      KC SANGAKKARA 
2            V KOHLI 
3         RT PONTING 
4      ST JAYASURIYA 
            ...      
495         DS AIREE 
496         DS SMITH 
497       JK KAMANDE 
498        IDS SMITH 
499     C KIESWETTER 
Name: Player_Names, Length: 450, dtype: object

In [32]:
df[["Start_Year","End_Year"]] = df[["Start_Year","End_Year"]].astype(object)

C:\Users\saura\AppData\Local\Temp\ipykernel_45160\3771160316.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["Start_Year","End_Year"]] = df[["Start_Year","End_Year"]].astype(object)


In [33]:
# Change multiple indexes at once
df.loc[[91, 370], 'Player_Names'] = ['Sean Williams', 'Stuart Williams']


In [34]:
df["Player_Names"].nunique()

450

In [35]:
df

,Player_Names,Country,Start_Year,End_Year,Matches,Innings,Not_Out,Total_Runs,Highest_Score,Batting_Average,Ball_Faced,Strike_Rate,100's,50's
0,SR Tendulkar,India,1989,2012,463,452,41,18426,200,44.83,21368,86.23,49,96
1,KC Sangakkara,Sri Lanka,2000,2015,404,380,41,14234,169,41.98,18048,78.86,25,93
2,V Kohli,India,2008,2025,302,290,45,14181,183,57.88,15192,93.34,51,74
3,RT Ponting,Australia,1995,2012,375,365,39,13704,164,42.03,17046,80.39,30,82
4,ST Jayasuriya,Sri Lanka,1989,2011,445,433,18,13430,189,32.36,14725,91.20,28,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,DS Airee,Nepal,2018,2025,64,60,7,1066,105,20.11,1472,72.41,1,4
496,DS Smith,West Indies,2003,2013,47,45,2,1059,107,24.62,1506,70.31,1,5
497,JK Kamande,Kenya,1999,2011,86,72,11,1055,74,17.29,1685,62.61,0,3
498,IDS Smith,New Zealand,1980,1992,98,77,16,1055,62,17.29,1061,99.43,0,3


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 450 entries, 0 to 499
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Player_Names     450 non-null    object 
 1   Country          450 non-null    object 
 2   Start_Year       450 non-null    object 
 3   End_Year         450 non-null    object 
 4   Matches          450 non-null    int64  
 5   Innings          450 non-null    int64  
 6   Not_Out          450 non-null    int64  
 7   Total_Runs       450 non-null    int64  
 8   Highest_Score    450 non-null    int64  
 9   Batting_Average  450 non-null    float64
 10  Ball_Faced       450 non-null    int64  
 11  Strike_Rate      450 non-null    float64
 12  100's            450 non-null    int64  
 13  50's             450 non-null    int64  
dtypes: float64(2), int64(8), object(4)
memory usage: 68.9+ KB


In [37]:
df = df[df["Country"] != "England/Ireland"]
df = df[df["Country"] != "Australia/South Africa"]
df = df[df["Country"] != "Australia/New Zealand"]
df

,Player_Names,Country,Start_Year,End_Year,Matches,Innings,Not_Out,Total_Runs,Highest_Score,Batting_Average,Ball_Faced,Strike_Rate,100's,50's
0,SR Tendulkar,India,1989,2012,463,452,41,18426,200,44.83,21368,86.23,49,96
1,KC Sangakkara,Sri Lanka,2000,2015,404,380,41,14234,169,41.98,18048,78.86,25,93
2,V Kohli,India,2008,2025,302,290,45,14181,183,57.88,15192,93.34,51,74
3,RT Ponting,Australia,1995,2012,375,365,39,13704,164,42.03,17046,80.39,30,82
4,ST Jayasuriya,Sri Lanka,1989,2011,445,433,18,13430,189,32.36,14725,91.20,28,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,DS Airee,Nepal,2018,2025,64,60,7,1066,105,20.11,1472,72.41,1,4
496,DS Smith,West Indies,2003,2013,47,45,2,1059,107,24.62,1506,70.31,1,5
497,JK Kamande,Kenya,1999,2011,86,72,11,1055,74,17.29,1685,62.61,0,3
498,IDS Smith,New Zealand,1980,1992,98,77,16,1055,62,17.29,1061,99.43,0,3


In [38]:
df["Country"].unique()

array(['India', 'Sri Lanka', 'Australia', 'Pakistan', 'South Africa',
       'West Indies', 'New Zealand', 'Bangladesh', 'England', 'Zimbabwe',
       'Ireland', 'Afghanistan', 'Scotland', 'Kenya', 'United States',
       'Namibia', 'Netherlands', 'Papua New Guinea', 'Canada',
       'United Arab Emirates', 'Nepal', 'Oman'], dtype=object)

# save csv file

In [39]:
df.to_csv("Cricket_Clean_Dataset.csv",index = False)